# Bull Call Spread

#### Construction of a Bull Call Spread
**Purchase** a **call** option at a **lower** strike price.
**Sell** a **call** option at a **higher** strike price.
Both Options should ideally have the same expiration date and should be written on the same underlying

#### Define a Call Option class in Python

In [2]:
class CallOption:
    def __init__(self, strike: float, premium: float):
        """
        Initialize a CallOption instance.

        :param strike: The strike price of the call option.
        :param premium: The premium paid for the call option.
        """
        self.strike = strike
        self.premium = premium
        self.delta = None
        self.gamma = None
        self.theta = None
        self.vega = None

    def set_greeks(self, delta: float = None, gamma: float = None, theta: float = None, vega: float = None):
        """
        Set the Greek values for the call option.

        :param delta: The delta value.
        :param gamma: The gamma value.
        :param theta: The theta value.
        :param vega: The vega value.
        """
        if delta is not None:
            self.delta = delta
        if gamma is not None:
            self.gamma = gamma
        if theta is not None:
            self.theta = theta
        if vega is not None:
            self.vega = vega

    def get_max_profit(self) -> str:
        """
        Calculate the maximum profit for the call option.

        :return: The maximum profit, which is theoretically infinite.
        """
        return "Infinite"

    def get_max_loss(self) -> float:
        """
        Calculate the maximum loss for the call option.

        :return: The maximum loss, which is the premium paid.
        """
        return self.premium

    def get_breakeven(self) -> float:
        """
        Calculate the breakeven point for the call option.

        :return: The breakeven point, which is the strike price plus the premium paid.
        """
        return self.strike + self.premium



##### Intantiate a two call options- one with the higher strike (likely lower premium) and the other with lower strike (likely higher premium)

In [3]:
# Define the strike prices and premiums for the call options
strike_high = 115
premium_high = 5

strike_low = 100
premium_low = 7

# Instantiate the call options
call_option_high = CallOption(strike=strike_high, premium=premium_high)
call_option_low = CallOption(strike=strike_low, premium=premium_low)

# Display the details of the call options
print(f"High Strike Call Option: Strike = {call_option_high.strike}, Premium = {call_option_high.premium}")
print(f"Low Strike Call Option: Strike = {call_option_low.strike}, Premium = {call_option_low.premium}")


High Strike Call Option: Strike = 115, Premium = 5
Low Strike Call Option: Strike = 100, Premium = 7


##### Determine the Maximum profit, Maximum loss and the Breakeven point

In [5]:
max_profit_bullcall = call_option_high.strike-call_option_low.strike-(call_option_low.premium-call_option_high.premium)
print (f"The maximum profit for this bull call spread strategy is {max_profit_bullcall}")
max_loss_bullcall = (call_option_low.premium-call_option_high.premium)
print (f"The maximum loss for this bull call option strategy is {max_loss_bullcall}")
breakeven_point = call_option_low.strike+(call_option_low.premium-call_option_high.premium)
print (f"The breakeven point for this bull call option strategy is {breakeven_point}")

The maximum profit for this bull call spread strategy is 13
The maximum loss for this bull call option strategy is 2
The breakeven point for this bull call option strategy is 102


Determine the Maximum profit, Maximum loss and the Breakeven point using a function

In [8]:
def calculate_metrics_bull_call(call_option_high, call_option_low):
    """
    Calculate the maximum profit for a bull call spread strategy.

    :param call_option_high: The call option with the higher strike price.
    :param call_option_low: The call option with the lower strike price.
    :return: The maximum profit from the bull call spread strategy.
    """
    max_profit = call_option_high.strike - call_option_low.strike - (call_option_low.premium - call_option_high.premium)
    max_loss = call_option_low.premium-call_option_high.premium
    breakeven = call_option_low.strike+(call_option_low.premium-call_option_high.premium)
    return max_profit, max_loss, breakeven

In [10]:
calculate_metrics_bull_call(call_option_high, call_option_low)

(13, 2, 102)

#### Analysing the Bull call spread using OpenAI (GPT4o)

In [22]:
from openai import OpenAI
# api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key='<Pasting the API key here is a Novice mistake. Use environment variables instead>')


In [15]:
from textwrap import dedent
financial_wizard_prompt = '''
    You are a super smart financial analyst who has passed CFA. You will be provided with financial derivative problems,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''
MODEL = "gpt-4o-2024-08-06"
def get_financial_derivative_solution(question):
    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system", 
            "content": dedent(financial_wizard_prompt)
        },
        {
            "role": "user", 
            "content": question
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "CFA_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
    )

    return response.choices[0].message

In [19]:
# Testing with an example question

# question = "The strike price of two call options are 115 and 100, the premiums are 5 and 7. What is the maximum profit, maximum loss and breakeven for Bull call spread"
question = f'''
the strike prices of the two call options of a bull call spread are {call_option_high.strike} and {call_option_low.strike}
The premiums for the corresponding call options are {call_option_high.premium} and {call_option_low.premium}
What is the maximum profit, maximum loss and breakeven for Bull call spread
'''
result = get_financial_derivative_solution(question) 

print(result.content)

{"steps":[{"explanation":"The first step is to identify the cost of entering the bull call spread. This involves buying a call option with a lower strike price and selling a call option with a higher strike price. We purchase the call option with a strike price of 100 at a premium of 7, and sell the call option with a strike price of 115 at a premium of 5.","output":"Net Premium Paid = Premium(buy) - Premium(sell) = 7 - 5 = 2"},{"explanation":"To calculate the maximum profit for a bull call spread, we need to calculate the difference between the strike prices and subtract the net premium paid. The strike price difference is 115 - 100, and the premium paid is 2.","output":"Maximum Profit = (Strike Price (Call Sold) - Strike Price (Call Bought)) - Net Premium Paid = (115 - 100) - 2 = 13"},{"explanation":"The maximum loss in a bull call spread is limited to the net premium paid. Here, it is the 2 premium paid per share for setting up the spread.","output":"Maximum Loss = Net Premium Paid 

In [20]:
import re
# Regular expression to extract the final_answer
match = re.search(r'"final_answer":"(.*?)"', result.content)

if match:
    final_answer = match.group(1)
    print(final_answer)
else:
    print("No final_answer found.")

Maximum Profit: 13, Maximum Loss: 2, Breakeven Point: 102


#### Conclusion
OpenAI's GPT4o gets the correct answers most of the time. The chain of reasoning is almost always correct, but it can sometimes get the arithmetic wrong